# School timetabling kotlin notebook

This Kotlin Notebook solves the school timetabling problem with [Timefold](https://timefold.ai], the open source planning solver AI.

![School timetabling input output](https://timefold.ai/docs/timefold-solver/latest/_images/quickstart/school-timetabling/schoolTimetablingInputOutput.png)

## Dependencies

Add the Timefold solver dependency:

In [74]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.3.0")


## Domain

Create the domain classes:

### Room

A school has rooms.

In [75]:
class Room {

    var name: String

    constructor(name: String) {
        this.name = name
    }

    override fun toString(): String = name

}

### Timeslot

A school timetable has timeslots.

In [76]:
import java.time.DayOfWeek
import java.time.LocalTime

class Timeslot {

    var dayOfWeek: DayOfWeek
    var startTime: LocalTime
    var endTime: LocalTime

    constructor(dayOfWeek: DayOfWeek, startTime: LocalTime, endTime: LocalTime) {
        this.dayOfWeek = dayOfWeek
        this.startTime = startTime
        this.endTime = endTime
    }

    override fun toString(): String = "$dayOfWeek $startTime"

}

### Lesson

Each lesson must be assigned to a timeslot and to a room by the solver.

In [77]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningVariable

@PlanningEntity
class Lesson {

    @PlanningId
    var id: Long? = null
    lateinit var subject: String
    lateinit var teacher: String
    lateinit var studentGroup: String

    @PlanningVariable
    var timeslot: Timeslot? = null
    @PlanningVariable
    var room: Room? = null

    // No-arg constructor required for Timefold
    constructor()

    constructor(id: Long, subject: String, teacher: String, studentGroup: String) {
        this.id = id
        this.subject = subject
        this.teacher = teacher
        this.studentGroup = studentGroup
    }


    override fun toString(): String = "$subject"

}

## Constraints

The solver must take into account hard and soft constraints:

In [78]:
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import java.time.Duration

class TimeTableConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Hard constraints
            roomConflict(constraintFactory),
            teacherConflict(constraintFactory),
            studentGroupConflict(constraintFactory),
            // Soft constraints
            teacherRoomStability(constraintFactory),
            teacherTimeEfficiency(constraintFactory),
            studentGroupSubjectVariety(constraintFactory)
        )
    }

    fun roomConflict(constraintFactory: ConstraintFactory): Constraint {
        // A room can accommodate at most one lesson at the same time.
        return constraintFactory
            // Select each pair of 2 different lessons ...
            .forEachUniquePair(
                Lesson::class.java,
                // ... in the same timeslot ...
                Joiners.equal(Lesson::timeslot),
                // ... in the same room ...
                Joiners.equal(Lesson::room)
            )
            // ... and penalize each pair with a hard weight.
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Room conflict");
    }

    fun teacherConflict(constraintFactory: ConstraintFactory): Constraint {
        // A teacher can teach at most one lesson at the same time.
        return constraintFactory
            .forEachUniquePair(
                Lesson::class.java,
                Joiners.equal(Lesson::timeslot),
                Joiners.equal(Lesson::teacher)
            )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Teacher conflict");
    }

    fun studentGroupConflict(constraintFactory: ConstraintFactory): Constraint {
        // A student can attend at most one lesson at the same time.
        return constraintFactory
            .forEachUniquePair(
                Lesson::class.java,
                Joiners.equal(Lesson::timeslot),
                Joiners.equal(Lesson::studentGroup)
            )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Student group conflict");
    }

    fun teacherRoomStability(constraintFactory: ConstraintFactory): Constraint {
        // A teacher prefers to teach in a single room.
        return constraintFactory
            .forEachUniquePair(
                Lesson::class.java,
                Joiners.equal(Lesson::teacher)
            )
            .filter { lesson1: Lesson, lesson2: Lesson -> lesson1.room !== lesson2.room }
            .penalize(HardSoftScore.ONE_SOFT)
            .asConstraint("Teacher room stability");
    }

    fun teacherTimeEfficiency(constraintFactory: ConstraintFactory): Constraint {
        // A teacher prefers to teach sequential lessons and dislikes gaps between lessons.
        return constraintFactory
            .forEach(Lesson::class.java)
            .join(Lesson::class.java,
                Joiners.equal(Lesson::teacher),
                Joiners.equal { lesson: Lesson -> lesson.timeslot?.dayOfWeek })
            .filter { lesson1: Lesson, lesson2: Lesson ->
                val between = Duration.between(
                    lesson1.timeslot?.endTime,
                    lesson2.timeslot?.startTime
                )
                !between.isNegative && between.compareTo(Duration.ofMinutes(30)) <= 0
            }
            .reward(HardSoftScore.ONE_SOFT)
            .asConstraint("Teacher time efficiency");
    }

    fun studentGroupSubjectVariety(constraintFactory: ConstraintFactory): Constraint {
        // A student group dislikes sequential lessons on the same subject.
        return constraintFactory
            .forEach(Lesson::class.java)
            .join(Lesson::class.java,
                Joiners.equal(Lesson::subject),
                Joiners.equal(Lesson::studentGroup),
                Joiners.equal { lesson: Lesson -> lesson.timeslot?.dayOfWeek })
            .filter { lesson1: Lesson, lesson2: Lesson ->
                val between = Duration.between(
                    lesson1.timeslot?.endTime,
                    lesson2.timeslot?.startTime
                )
                !between.isNegative && between.compareTo(Duration.ofMinutes(30)) <= 0
            }
            .penalize(HardSoftScore.ONE_SOFT)
            .asConstraint("Student group subject variety");
    }

}

### TimeTable

The timetable class represents a single dataset, typically a single school:

In [79]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

@PlanningSolution
class TimeTable {

    @ProblemFactCollectionProperty
    @ValueRangeProvider
    lateinit var timeslots: List<Timeslot>
    @ProblemFactCollectionProperty
    @ValueRangeProvider
    lateinit var rooms: List<Room>
    @PlanningEntityCollectionProperty
    lateinit var lessons: List<Lesson>

    @PlanningScore
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() {}

    constructor(timeslots: List<Timeslot>, rooms: List<Room>, lessons: List<Lesson>) {
        this.timeslots = timeslots
        this.rooms = rooms
        this.lessons = lessons
    }

}

## Data generator

Generate some data for a small school timetable:

In [80]:
fun generateDemoData(): TimeTable {
    val timeslots: MutableList<Timeslot> = mutableListOf(
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(8, 30), LocalTime.of(9, 30)),
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(9, 30), LocalTime.of(10, 30)),
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(10, 30), LocalTime.of(11, 30)),
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(13, 30), LocalTime.of(14, 30)),
            Timeslot(DayOfWeek.MONDAY, LocalTime.of(14, 30), LocalTime.of(15, 30)),
    
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(8, 30), LocalTime.of(9, 30)),
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(9, 30), LocalTime.of(10, 30)),
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(10, 30), LocalTime.of(11, 30)),
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(13, 30), LocalTime.of(14, 30)),
            Timeslot(DayOfWeek.TUESDAY, LocalTime.of(14, 30), LocalTime.of(15, 30)))
    
    
    val rooms: MutableList<Room> = mutableListOf(
            Room("Room A"),
            Room("Room B"),
            Room("Room C"))
    
    var nextId: Long = 0
    val lessons: MutableList<Lesson> = mutableListOf(
            Lesson(nextId++, "Math", "A. Turing", "9th grade"),
            Lesson(nextId++, "Math", "A. Turing", "9th grade"),
            Lesson(nextId++, "Physics", "M. Curie", "9th grade"),
            Lesson(nextId++, "Chemistry", "M. Curie", "9th grade"),
            Lesson(nextId++, "Biology", "C. Darwin", "9th grade"),
            Lesson(nextId++, "History", "I. Jones", "9th grade"),
            Lesson(nextId++, "English", "I. Jones", "9th grade"),
            Lesson(nextId++, "English", "I. Jones", "9th grade"),
            Lesson(nextId++, "Spanish", "P. Cruz", "9th grade"),
            Lesson(nextId++, "Spanish", "P. Cruz", "9th grade"),
            Lesson(nextId++, "Math", "A. Turing", "10th grade"),
            Lesson(nextId++, "Math", "A. Turing", "10th grade"),
            Lesson(nextId++, "Math", "A. Turing", "10th grade"),
            Lesson(nextId++, "Physics", "M. Curie", "10th grade"),
            Lesson(nextId++, "Chemistry", "M. Curie", "10th grade"),
            Lesson(nextId++, "French", "M. Curie", "10th grade"),
            Lesson(nextId++, "Geography", "C. Darwin", "10th grade"),
            Lesson(nextId++, "History", "I. Jones", "10th grade"),
            Lesson(nextId++, "English", "P. Cruz", "10th grade"),
            Lesson(nextId++, "Spanish", "P. Cruz", "10th grade"))
    return TimeTable(timeslots, rooms, lessons)
}

## Visualize the result

Print the timetable to the console:

In [81]:
fun printTimeTable(timeTable: TimeTable) {
    println("")

    val lessonMap = timeTable.lessons.groupBy { lesson -> Pair(lesson.timeslot, lesson.room) }

    print("|            ")
    for (room in timeTable.rooms) {
        print("| " + String.format("%-10s", room.name) + " ")    
    }
    println("|") 
    println("|" + "------------|".repeat(timeTable.rooms.size + 1))  
    
    for (timeslot in timeTable.timeslots) {
        print("| " + String.format("%-10s", "" + timeslot.dayOfWeek.toString().subSequence(0, 3) + " " + timeslot.startTime) + " ")
        for (room in timeTable.rooms) {
            print("| ")   
            val cellLessons = lessonMap.get(Pair(timeslot, room)) ?: emptyList()
            print(String.format("%-10s", cellLessons.map { lesson -> lesson.subject }.joinToString(", ")))
            print(" ")
        }
        println("|")
        print("| " + String.format("%-10s", "") + " ")
        for (room in timeTable.rooms) {
            print("| ")   
            val cellLessons = lessonMap.get(Pair(timeslot, room)) ?: emptyList()
            print(String.format("%-10s", cellLessons.map { lesson -> lesson.teacher }.joinToString(", ")))
            print(" ")
        }
        println("|")
        print("| " + String.format("%-10s", "") + " ")
        for (room in timeTable.rooms) {
            print("| ")   
            val cellLessons = lessonMap.get(Pair(timeslot, room)) ?: emptyList()
            print(String.format("%-10s", cellLessons.map { lesson -> lesson.studentGroup }.joinToString(", ")))
            print(" ")
        }
        println("|") 
        println("|" + "------------|".repeat(timeTable.rooms.size + 1)) 
    }
    val unassignedLessons = lessonMap.get(Pair(null, null))
    if (unassignedLessons != null && unassignedLessons.isNotEmpty()) {
        println("")
        println("Unassigned lessons")
        for (lesson: Lesson in unassignedLessons) {
            println("  " + lesson.subject + " - " + lesson.teacher + " - " + lesson.studentGroup)
        }
    }
}

## Solve it

Configure and run the solver:

In [82]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import ai.timefold.solver.core.api.solver.SolutionManager
import ai.timefold.solver.core.api.solver.SolverManager

val solverFactory: SolverFactory<TimeTable> = SolverFactory.create(SolverConfig()
        .withSolutionClass(TimeTable::class.java)
        .withEntityClasses(Lesson::class.java)
        .withConstraintProviderClass(TimeTableConstraintProvider::class.java)
        // The solver runs only for 5 seconds on this small dataset.
        // It's recommended to run for at least 5 minutes ("5m") otherwise.
        .withTerminationSpentLimit(Duration.ofSeconds(5)))

println("Loading the problem ...")
val problem: TimeTable = generateDemoData()

println("Solving the problem ...")
val solver: Solver<TimeTable> = solverFactory.buildSolver()
val solution: TimeTable = solver.solve(problem)

println("Visualizing the solution ...")
printTimeTable(solution)


Loading the problem ...
Solving the problem ...
Visualizing the solution ...

|            | Room A     | Room B     | Room C     |
|------------|------------|------------|------------|
| MON 08:30  | Math       | Spanish    |            |
|            | A. Turing  | P. Cruz    |            |
|            | 10th grade | 9th grade  |            |
|------------|------------|------------|------------|
| MON 09:30  | Math       | English    |            |
|            | A. Turing  | P. Cruz    |            |
|            | 9th grade  | 10th grade |            |
|------------|------------|------------|------------|
| MON 10:30  | Math       | Spanish    |            |
|            | A. Turing  | P. Cruz    |            |
|            | 10th grade | 9th grade  |            |
|------------|------------|------------|------------|
| MON 13:30  |            | English    | Geography  |
|            |            | I. Jones   | C. Darwin  |
|            |            | 9th grade  | 10th grade |
|---

To learn more, visit [timefold.ai](https://timefold.ai).